In [1]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb



#testar com todos  'EfficientNetB0'
# rgb

In [3]:
# Criando função para calcular as métricas
def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }

In [5]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    elif scale == 'YcrCb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    else:
        raise ValueError("Escala de cor não suportada.")

In [6]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor
def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário

                # Converta para a escala de cor desejada
                img = convert_color_scale(img, color_scale)

                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal

                images.append(img)
    return np.array(images)

In [5]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"
normal_dir = os.path.join(data_dir, 'Non_Cancerous')
cancer_dir = os.path.join(data_dir, 'Serous')
img_extensions = ['.jpg', '.jpeg', '.png']


In [6]:
# Criar um DataFrame para armazenar os resultados
# columns = ['Modelo', 'Acuracia', 'Sensibilidade', 'Especificidade', 'F-Score', 'AUC-ROC']
# df_metrics = pd.DataFrame(columns=columns)


In [7]:
# Carregar o modelo InceptionV3 pré-treinado
base_model =VGG16(weights='imagenet', include_top=False)     # <----------------------------------------------------------

# Extrair características usando o modelo VGG16
def extract_features(model, X):
    features = model.predict(X)
    return features.reshape((features.shape[0], -1))

In [8]:
cores = ['hsv', 'rgb', 'grayscale', 'lab', 'luv', 'xyz']  # <----------------------------------------------------------
df_all_metrics = pd.DataFrame()
for esquema in cores:
    # Carregamento de imagens e conversão para XYZ
    normal_images = load_images(normal_dir, esquema,img_extensions)        # <----------------------------------------------------------
    cancer_images = load_images(cancer_dir, esquema,img_extensions)        # hsv<----------------------------------------------------------


    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_images = extract_features(base_model, all_images)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

    X_train = X_train / 255.0
    X_test = X_test / 255.0






    # Para o Random Forest
    rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    metrics_rf = get_metrics(y_test, y_pred_rf)
    print(f"Métricas para {esquema} (Random Forest):")
    print(metrics_rf)


    df_metrics_rf = pd.DataFrame([metrics_rf.values()], columns=metrics_rf.keys())
    df_metrics_rf.insert(0, 'esquemaCor', esquema)



    # Concatenar todos os DataFrames
    df_all_metrics = pd.concat([df_all_metrics, df_metrics_rf], ignore_index=True)

    # Salvar o DataFrame em um arquivo Excel
df_all_metrics.to_excel('resulEscquemaCor.xlsx', index=False)   # <----------------------------------------------------------

df_all_metrics


Métricas para hsv (Random Forest):
{'accuracy': 0.85, 'specificity': 0.8571428571428571, 'recall': 0.8421052631578947, 'precision': 0.8421052631578947, 'f1-score': 0.8421052631578947, 'kappa': 0.6992481203007519, 'auc-roc': 0.849624060150376}
Métricas para rgb (Random Forest):
{'accuracy': 0.9, 'specificity': 0.9523809523809523, 'recall': 0.8421052631578947, 'precision': 0.9411764705882353, 'f1-score': 0.8888888888888888, 'kappa': 0.7984886649874056, 'auc-roc': 0.8972431077694235}
Métricas para grayscale (Random Forest):
{'accuracy': 0.925, 'specificity': 0.9523809523809523, 'recall': 0.8947368421052632, 'precision': 0.9444444444444444, 'f1-score': 0.918918918918919, 'kappa': 0.8492462311557789, 'auc-roc': 0.9235588972431078}
Métricas para lab (Random Forest):
{'accuracy': 0.825, 'specificity': 0.9523809523809523, 'recall': 0.6842105263157895, 'precision': 0.9285714285714286, 'f1-score': 0.7878787878787878, 'kappa': 0.6446700507614214, 'auc-roc': 0.8182957393483709}
Métricas para luv (

,esquemaCor,accuracy,specificity,recall,precision,f1-score,kappa,auc-roc
0,hsv,0.850,0.857143,0.842105,0.842105,0.842105,0.699248,0.849624
1,rgb,0.900,0.952381,0.842105,0.941176,0.888889,0.798489,0.897243
2,grayscale,0.925,0.952381,0.894737,0.944444,0.918919,0.849246,0.923559
3,lab,0.825,0.952381,0.684211,0.928571,0.787879,0.644670,0.818296
4,luv,0.850,0.904762,0.789474,0.882353,0.833333,0.697733,0.847118
5,xyz,0.900,0.952381,0.842105,0.941176,0.888889,0.798489,0.897243
